### Methodology

1- First we import the necessary libraries:

In [1]:
import pandas as pd
import requests
import folium
import json
from geopy.geocoders import Nominatim #to convert an address into latitude and longitude
from pandas.io.json import json_normalize #to tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans

2-We extract the information regarding the neighborhoods of Toronto city using a Wikipedia page:
https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
and then we transform it to a Pandas DataFrame and then we reset the index:

In [2]:
wiki='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_page=requests.get(wiki)
toronto=pd.read_html(wiki_page.content,header=0)[0]
toronto=toronto.loc[toronto['Borough']!='Not assigned']
toronto.reset_index(inplace=True,drop=True)
toronto.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


3- Now to determine the location of neighborhoods, first we have to retrive the latitude and longitude of each neighborhood,using the information provided on https://cocl.us/Geospatial_data:

In [3]:
path='https://cocl.us/Geospatial_data'
toronto_neighborhoods_lat_lng=pd.read_csv(path)
toronto_neighborhoods_lat_lng.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


4- Now we have to combine these two dataframes to obtain the full information of each neighborhood:

In [4]:
toronto=toronto.merge(toronto_neighborhoods_lat_lng.set_index('Postal Code'),on='Postal Code')
toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


5- Now we use Folium library to draw the map of Toronto City, and then we locate each neighborhood on the map:

In [5]:
toronto_lat=43.651070
toronto_lng=-79.347015
toronto_map=folium.Map(location=[toronto_lat,toronto_lng],zoom_start=11)

for i in range(toronto.shape[0]):
    lat=toronto['Latitude'][i]
    lng=toronto['Longitude'][i]
    neigh=toronto['Neighbourhood'][i]
    bor=toronto['Borough'][i]
    label=bor+neigh
    folium.CircleMarker([lat,lng],
                       radius=5,
                       fill=True,
                       color='blue',
                       popup=label,
                       fill_color='blue',
                       fill_opacity=0.7).add_to(toronto_map)
toronto_map

6-Now it is time to retrieve the information regarding to the restaurants around each neighborhood:
first we define a function to get the information from www.foursquare.com for a given location:

In [20]:
client_id = 'LF5DZSYWSOKCMXWGRSW5FXW15T1EP4BX1A3YLAYVMOUZQNGI'
client_secret = 'OZILLTCWZC2HURLAIAKBPHMJNRMMGGQNBWQ4KKJUX0QH3CTV'
version = '20201014'
limit = 100
lat=toronto['Latitude'][0]
lng=toronto['Longitude'][0]
query='restaurant'
radius=500
url='https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
                                                client_id,client_secret,lat,lng,version,query,radius,limit)

7-First lets try the information regarding the first neighborhood in order to become acquainted with the information exist in the JSON file:

In [21]:
request=requests.get(url).json()
request

{'meta': {'code': 200, 'requestId': '5f8bf027dc3fa014e6f584e8'},
 'response': {'warning': {'text': 'There aren\'t a lot of results for "restaurant." Try something more general, reset your filters, or expand the search area.'},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'query': 'restaurant',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.757758604500005,
    'lng': -79.32343823984928},
   'sw': {'lat': 43.7487585955, 'lng': -79.33587476015072}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e6696b6d16433b9ffff47c3',
       'name': 'KFC',
       'location': {'lat': 43.75438666345904,
        'lng': -79.3330206627504,
        'labeledLatLngs': [{'label':

8- So lets define a function to extract the categories of restaurant in each search:

In [22]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']        

In [24]:
import json
from pandas.io.json import json_normalize

venues = request['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) 

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

C:\Users\iepgh\anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,name,categories,lat,lng
0,KFC,Fast Food Restaurant,43.754387,-79.333021
1,Bella Vita Catering & Private Chef Service,BBQ Joint,43.756651,-79.331524


In [30]:
def getNearbyRestaurants(names, latitudes, longitudes, radius=500):
    
    restaurants_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
    
        url='https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
                                                client_id,client_secret,lat,lng,version,'restaurant',radius,limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        restaurants_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in restaurants_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [31]:
Toronto_restaurants = getNearbyRestaurants(names=toronto['Neighbourhood'],
                                   latitudes=toronto['Latitude'],
                                   longitudes=toronto['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

9- Lets take a look at the Toronto_restaurants dataframe:

In [32]:
Toronto_restaurants.head()

,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
1,Parkwoods,43.753259,-79.329656,Bella Vita Catering & Private Chef Service,43.756651,-79.331524,BBQ Joint
2,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
3,Victoria Village,43.725882,-79.315572,The Frig,43.727051,-79.317418,French Restaurant
4,Victoria Village,43.725882,-79.315572,Pizza Nova,43.725824,-79.312860,Pizza Place


In [45]:
Toronto_restaurants.rename(columns={'Venue':'Restaurant Name','Venue Category':'Restaurant Category'},inplace=True)

In [46]:
Toronto_restaurants.head()

,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Restaurant Name,Venue Latitude,Venue Longitude,Restaurant Category
0,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
1,Parkwoods,43.753259,-79.329656,Bella Vita Catering & Private Chef Service,43.756651,-79.331524,BBQ Joint
2,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
3,Victoria Village,43.725882,-79.315572,The Frig,43.727051,-79.317418,French Restaurant
4,Victoria Village,43.725882,-79.315572,Pizza Nova,43.725824,-79.312860,Pizza Place


10- Now we will see the total number of each Restaurant Category in Toronto City

In [47]:
# one hot encoding
toronto_onehot = pd.get_dummies(Toronto_restaurants[['Restaurant Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = Toronto_restaurants['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot =toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Afghan Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Belgian Restaurant,...,Steakhouse,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
toronto_onehot.shape

(1636, 92)

In [51]:
toronto_grouped=toronto_onehot.groupby('Neighbourhood').mean()

In [52]:
toronto_grouped.reset_index(inplace=True)

In [53]:
toronto_grouped.head()

,Neighbourhood,Afghan Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Belgian Restaurant,...,Steakhouse,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint
0,Agincourt,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.111111,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.047619,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.095238,0.0,0.0,0.0,0.095238,0.0,0.0,0.0,0.0


In [54]:
def return_most_common_restaurants(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [57]:
import numpy as np
num_top_restaurants = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_restaurants):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_restaurants_sorted = pd.DataFrame(columns=columns)
neighborhoods_restaurants_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_restaurants_sorted.iloc[ind, 1:] = return_most_common_restaurants(toronto_grouped.iloc[ind, :], num_top_restaurants)

neighborhoods_restaurants_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Breakfast Spot,Chinese Restaurant,Latin American Restaurant,Fast Food Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant
1,"Alderwood, Long Branch",Pizza Place,Restaurant,Burger Joint,Wings Joint,Deli / Bodega,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Restaurant,Pizza Place,Fried Chicken Joint,Deli / Bodega,Chinese Restaurant,Diner,Sushi Restaurant,Middle Eastern Restaurant,Eastern European Restaurant,Dim Sum Restaurant
3,Bayview Village,Chinese Restaurant,Japanese Restaurant,Café,Wings Joint,Falafel Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
4,"Bedford Park, Lawrence Manor East",Pizza Place,Italian Restaurant,Restaurant,Thai Restaurant,Sushi Restaurant,Comfort Food Restaurant,Greek Restaurant,Indian Restaurant,American Restaurant,Japanese Restaurant


11- Now that we know the most common types of restaurants in each neighborhood, we should cluster the neighborhoods based on the restaurant types in the area:

In [59]:
from sklearn.cluster import KMeans

kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_[0:30]

array([1, 0, 1, 1, 1, 1, 0, 1, 4, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4,
       1, 1, 1, 1, 2, 1, 1, 1])

12- Know it we add the Cluster label of each neighborhood:

In [62]:
neighborhoods_restaurants_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_restaurants_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,4.0,Fast Food Restaurant,BBQ Joint,Wings Joint,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Burger Joint,Portuguese Restaurant,French Restaurant,Pizza Place,Wings Joint,Deli / Bodega,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1.0,Restaurant,Café,Bakery,Breakfast Spot,Seafood Restaurant,French Restaurant,Greek Restaurant,Thai Restaurant,Food Truck,Asian Restaurant
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1.0,Vietnamese Restaurant,Fast Food Restaurant,Hot Dog Joint,Diner,Dim Sum Restaurant,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1.0,Sushi Restaurant,Restaurant,Diner,Middle Eastern Restaurant,Burrito Place,Café,Persian Restaurant,Creperie,Deli / Bodega,Wings Joint
